# Saving from Disaster using Python

Import the neccessary libraries (the chained_assignment command is used to suppres some **pandas** warning in the terminal).

In [1]:
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
pd.options.mode.chained_assignment = None

import datetime
import os
os.chdir("/home/numan/Desktop/Titanic/")

## Define Helper Functions

In [2]:
def testPrediction(pred, soln='solution.csv', soln_col="Survived", soln_csv=True, formatted=True):
    if soln_csv:
        soln=pd.read_csv(soln)[soln_col].values
    p = np.mean(soln==pred)
    if formatted:
        return "Prediction Accuracy: {:.2%}".format(p)
    else:
        return p


def saveCSV(pred, name):
    global test
    PassengerId =np.array(test["PassengerId"]).astype(int)
    my_solution = pd.DataFrame(pred, PassengerId, columns = ["Survived"])
    my_solution.to_csv("solution_output/"+name+" - "+datetime.datetime.now().strftime("%s")+".csv", 
                       index_label = ["PassengerId"])


## Load and clean-up the **training** data. 

Convert the _male_ to 0 and _female_ to 1. Similarly convert the _embarkment_ information from *S*, *C* and *Q* to 0, 1 and 2 respectively. Fill the missing embarkment information to **S**, the "mode" of the rest of information.

In [3]:
train = pd.read_csv("train.csv")

train["Sex"][train["Sex"] == "male"] = 0
train["Sex"][train["Sex"] == "female"] = 1

train["Embarked"] = train["Embarked"].fillna("S")
train["Embarked"][train["Embarked"] == "S"] = 0
train["Embarked"][train["Embarked"] == "C"] = 1
train["Embarked"][train["Embarked"] == "Q"] = 2

Fill the missing Age data using the "mean" (not a great idea, I admit, doesn;t help too much, uh huh). Create new features called **child** and **family_size**. (Aha!!! this is what they call *feature engineering*) 

In [4]:
train["Age"] = train["Age"].fillna(-1)
train["Child"] = float('NaN')
train["Child"][train["Age"]<0]=-1
train["Child"][train["Age"]<18]=1
train["Child"][train["Age"]>=18]=0

train["family_size"] = 1+train["SibSp"]+train["Parch"]

train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Child,family_size
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,23.600640,0.523008,0.381594,32.204208,0.325477,1.904602
std,257.353842,0.486592,0.836071,17.867496,1.102743,0.806057,49.693429,0.468816,1.613459
min,1.000000,0.000000,1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,223.500000,0.000000,2.000000,6.000000,0.000000,0.000000,7.910400,0.000000,1.000000
50%,446.000000,0.000000,3.000000,24.000000,0.000000,0.000000,14.454200,0.000000,1.000000
75%,668.500000,1.000000,3.000000,35.000000,1.000000,0.000000,31.000000,1.000000,2.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200,1.000000,11.000000


Now load and clean-up the **test** data. It has 152nd value missing in the *fare* column, just fill it with "median".

In [5]:
test = pd.read_csv("test.csv")
test["Sex"][test["Sex"] == "male"] = 0
test["Sex"][test["Sex"] == "female"] = 1

train["Embarked"] = train["Embarked"].fillna(-1)
test["Embarked"][test["Embarked"] == "S"] = 0
test["Embarked"][test["Embarked"] == "C"] = 1
test["Embarked"][test["Embarked"] == "Q"] = 2

In [6]:
test.Fare[152] = test.Fare.median()
test["Age"] = test["Age"].fillna(-1)
test["Child"] = float('NaN')
test["Child"][train["Age"]<0]=-1
test["Child"][test["Age"]<18]=1
test["Child"][test["Age"]>=18]=0

test["family_size"] = 1+test["SibSp"]+test["Parch"]

#test

Ok Let us set the target variable, **survived**.

In [7]:
target = train["Survived"].values

## Decision Tree Classifier

Now train the classifier, **DecisionTree**, and print the effectiveness(score) of the classifier on the _training_ data.

In [8]:
features = train[["Pclass", "Sex", "Age", "Fare", "SibSp", "Parch", "family_size", "Embarked"]].values
clasfr = tree.DecisionTreeClassifier()
d_tree = clasfr.fit(features,target)
print("Classifier Score: {:.2%}".format(d_tree.score(features,target))) 

Classifier Score: 98.20%


Now apply the trained classifier to the testdata and save it to *D-tree.csv*. Load the actual Solution and compare the predicted results. Prepare the output for Kaggle submission.

In [9]:
test_features = test[["Pclass", "Sex", "Age", "Fare", "SibSp", "Parch", "family_size", "Embarked"]].values
my_prediction = d_tree.predict(test_features)
saveCSV(my_prediction, "D-tree")
testPrediction(my_prediction)

'Prediction Accuracy: 70.10%'

**Submission Score**: 0.68900

## Random Forest Classifier

Now train the classifier, **RandomForest**, and print the effectiveness(score) of the classifier on the _training_ data.

In [10]:
features = train[["Pclass", "Sex", "Age", "Fare", "SibSp", "Parch", "family_size", "Embarked"]].values
clasfr = RandomForestClassifier(max_depth = 10, min_samples_split=2, n_estimators = 100, random_state = 1)
d_forest = clasfr.fit(features, target)
print("Classifier Score: {:.2%}".format(d_tree.score(features,target))) 

Classifier Score: 98.20%


In [11]:
test_features = test[["Pclass", "Sex", "Age", "Fare", "SibSp", "Parch", "family_size", "Embarked"]].values
my_prediction2 = d_forest.predict(test_features)
saveCSV(my_prediction2, "D-forest")
testPrediction(my_prediction2)

'Prediction Accuracy: 74.88%'

**Submission Score**: 0.76077

The [next step](http://localhost:8888/notebooks/Desktop/Titanic/Titanic%20Version%202%20-%20ahmedbesbes.com.ipynb) is to try [Ahmed Besbes's Tutorial](http://ahmedbesbes.com/how-to-score-08134-in-titanic-kaggle-challenge.html) in Python.

Another [option](https://www.kaggle.com/rajatshah/scikit-learn-ml-from-start-to-finish) to try.